In [74]:
! pip install pyfpgrowth

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: syncthing-gtk v0.9.4.4-ds-git20220108-9023143f8b93 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of syncthing-gtk or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [75]:
import pandas as pd

itemset_df = pd.read_csv('data/itemset.csv')

itemset_df.head()

superhost_itemset_df = itemset_df[itemset_df['host_is_superhost']].drop('host_is_superhost', axis = 1)
not_superhost_itemset_df = itemset_df[~itemset_df['host_is_superhost']].drop('host_is_superhost', axis = 1)

itemset_df

,host_is_superhost,host_has_profile_pic,host_identity_verified,host_has_email_verification,host_has_phone_verification,host_has_work_email_verification,host_has_photographer_verification,host_response_time_a_few_days_or_more,host_response_time_within_a_day,host_response_time_within_a_few_hours,...,host_response_rate>=50%,host_acceptance_rate>=50%,host_response_rate>=60%,host_acceptance_rate>=60%,host_response_rate>=70%,host_acceptance_rate>=70%,host_response_rate>=80%,host_acceptance_rate>=80%,host_response_rate>=90%,host_acceptance_rate>=90%
0,True,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
2,True,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
3,False,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
4,False,True,False,False,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29867,False,True,True,False,False,False,False,False,True,False,...,True,False,True,False,True,False,True,False,False,False
29868,False,True,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
29869,False,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
29870,False,True,True,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True


In [76]:
def get_transaction(row):
    transaction = []
    
    for label, flag in row.items():
        if flag: transaction.append(label)
    
    return transaction

superhost_transactions = list(superhost_itemset_df.apply(get_transaction, axis = 1))
not_superhost_transactions = list(not_superhost_itemset_df.apply(get_transaction, axis = 1))

#transactions

In [77]:
def simplify_itemset(itemset):
    max_response_rate = -1
    max_acceptance_rate = -1

    for item in itemset:
        if item.startswith('host_response_rate>='): max_response_rate = max(max_response_rate, int(item[20:-1]))
        if item.startswith('host_acceptance_rate>='): max_acceptance_rate = max(max_acceptance_rate, int(item[22:-1]))
    
    new_itemset = [item for item in itemset if not item.startswith('host_response_rate>=') and not item.startswith('host_acceptance_rate>=')]
        
    if max_response_rate > -1: new_itemset += [f'host_response_rate>={max_response_rate}%']
    if max_acceptance_rate > -1: new_itemset += [f'host_acceptance_rate>={max_response_rate}%']

    

    return tuple(new_itemset)

In [78]:
import pyfpgrowth as fp_growth

support_threshold = 0.8

superhost_patterns = fp_growth.find_frequent_patterns(superhost_transactions, len(superhost_transactions) * support_threshold)
not_superhost_patterns = fp_growth.find_frequent_patterns(not_superhost_transactions, len(not_superhost_transactions) * support_threshold)

In [79]:
superhost_patterns = {simplify_itemset(itemset): support for itemset, support in superhost_patterns.items()}
not_superhost_patterns = {simplify_itemset(itemset): support for itemset, support in not_superhost_patterns.items()}

In [80]:
unique_superhost_patterns = {itemset: support for itemset, support in superhost_patterns.items() if itemset not in not_superhost_patterns}
unique_not_superhost_patterns = {itemset: support for itemset, support in not_superhost_patterns.items() if itemset not in superhost_patterns}

print(f'Unique superhost patterns: {len(unique_superhost_patterns)}')
print(f'Unique not superhost patterns: {len(unique_not_superhost_patterns)}')

Unique superhost patterns: 78
Unique not superhost patterns: 2


In [81]:
for k in unique_superhost_patterns: print(k)

('host_response_time_within_an_hour', 'host_acceptance_rate>=-1%')
('host_response_time_within_an_hour', 'host_response_rate>=80%', 'host_acceptance_rate>=80%')
('host_response_time_within_an_hour', 'host_response_rate>=70%', 'host_acceptance_rate>=70%')
('host_response_time_within_an_hour', 'host_response_rate>=0%', 'host_acceptance_rate>=0%')
('host_response_time_within_an_hour', 'host_response_rate>=10%', 'host_acceptance_rate>=10%')
('host_response_time_within_an_hour', 'host_response_rate>=20%', 'host_acceptance_rate>=20%')
('host_response_time_within_an_hour', 'host_response_rate>=30%', 'host_acceptance_rate>=30%')
('host_response_time_within_an_hour', 'host_response_rate>=40%', 'host_acceptance_rate>=40%')
('host_response_time_within_an_hour', 'host_response_rate>=50%', 'host_acceptance_rate>=50%')
('host_response_time_within_an_hour', 'host_response_rate>=60%', 'host_acceptance_rate>=60%')
('host_response_time_within_an_hour', 'host_response_rate>=90%')
('host_response_time_wit

In [86]:
for k in unique_not_superhost_patterns: print(k)

('host_has_profile_pic', 'host_identity_verified')
('host_has_profile_pic',)


In [82]:
count = 0

for itemset, frequency in sorted(patterns.items(), key = lambda pair: pair[1], reverse = True):
    print(f'{frequency}: {itemset}')
    count += 1

    if count > 10: break

9521: ('host_response_rate>=0%',)
9521: ('host_acceptance_rate>=0%',)
9521: ('host_acceptance_rate>=0%', 'host_response_rate>=0%')
9521: ('host_acceptance_rate>=10%',)
9521: ('host_acceptance_rate>=0%', 'host_acceptance_rate>=10%')
9521: ('host_acceptance_rate>=10%', 'host_response_rate>=0%')
9521: ('host_acceptance_rate>=0%', 'host_acceptance_rate>=10%', 'host_response_rate>=0%')
9521: ('host_acceptance_rate>=20%',)
9521: ('host_acceptance_rate>=10%', 'host_acceptance_rate>=20%')
9521: ('host_acceptance_rate>=0%', 'host_acceptance_rate>=20%')
9521: ('host_acceptance_rate>=20%', 'host_response_rate>=0%')


In [83]:
len(patterns)

524280

In [84]:
confidence_threshold = 0.6
rules = fp_growth.generate_association_rules(unique_superhost_patterns, confidence_threshold)

In [85]:
print(rules)

{('host_response_rate>=90%',): (('host_has_profile_pic',), 0.9946698575002719), ('host_has_profile_pic', 'host_response_rate>=80%'): (('host_identity_verified',), 0.8921295303452713), ('host_has_profile_pic', 'host_response_rate>=90%'): (('host_identity_verified',), 0.8924978127734033), ('host_identity_verified', 'host_response_rate>=90%'): (('host_has_profile_pic',), 0.9964590964590965), ('host_has_profile_pic', 'host_identity_verified', 'host_response_rate>=90%'): (('host_acceptance_rate>=90%',), 0.9987746599681412), ('host_identity_verified', 'host_response_rate>=80%'): (('host_has_profile_pic',), 0.9965265301233681), ('host_has_profile_pic', 'host_identity_verified', 'host_response_rate>=80%'): (('host_acceptance_rate>=80%',), 0.9987980769230769), ('host_identity_verified', 'host_response_rate>=70%'): (('host_has_profile_pic',), 0.9964379007361672), ('host_has_profile_pic', 'host_identity_verified', 'host_response_rate>=70%'): (('host_acceptance_rate>=70%',), 0.9988083889418494), (